In [2]:
import trimesh
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from matplotlib import pyplot as plt

C:\Users\Arif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [46]:
# Load the tooth STL model using Trimesh
mesh_trimesh = trimesh.load_mesh("stlFiles/sinifbirNumaraAlti.stl")

# Get vertices, faces, and normals
vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)

dent_mesh = o3d.geometry.TriangleMesh()
dent_mesh.vertices = o3d.utility.Vector3dVector(vertices)
dent_mesh.triangles = o3d.utility.Vector3iVector(faces)
dent_mesh.compute_vertex_normals()
# Compute Mean Curvature using Trimesh
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, radius=2)


In [47]:
def extract_largest_cavity(vertices, faces, cavity_indices):
    # Get unique cavity indices
    unique_cavity_indices = np.unique(cavity_indices)
    
    # Find faces that have all vertices in cavity_indices
    cavity_face_mask = np.isin(faces.ravel(), unique_cavity_indices).reshape(faces.shape)
    cavity_face_indices = np.where(np.all(cavity_face_mask, axis=1))[0]
    cavity_faces = faces[cavity_face_indices]
    
    # Create adjacency matrix for connected component analysis
    edges = set()
    for face in cavity_faces:
        edges.add((face[0], face[1]))
        edges.add((face[1], face[2]))
        edges.add((face[2], face[0]))
    
    # Create sparse adjacency matrix
    row, col = zip(*edges)
    row = np.array(row)
    col = np.array(col)
    data = np.ones_like(row)
    
    # Create sparse matrix with size equal to total vertices 
    # (will be pruned to cavity vertices later)
    adj_matrix = csr_matrix((data, (row, col)), shape=(vertices.shape[0], vertices.shape[0]))
    
    # Find connected components
    n_components, labels = connected_components(csgraph=adj_matrix, directed=False)
    
    # Count vertices in each component
    component_sizes = np.zeros(n_components, dtype=int)
    for label in labels[unique_cavity_indices]:
        component_sizes[label] += 1
    
    # Find the largest component
    largest_component = np.argmax(component_sizes)
    
    # Get vertices from the largest component
    largest_cavity_indices = np.where(labels == largest_component)[0]
    largest_cavity_indices = np.intersect1d(largest_cavity_indices, unique_cavity_indices)
    
    # Create index mapping for new mesh
    index_map = np.zeros(len(vertices), dtype=int)
    for i, idx in enumerate(largest_cavity_indices):
        index_map[idx] = i
    
    # Get faces for largest component
    largest_face_mask = np.isin(cavity_faces.ravel(), largest_cavity_indices).reshape(cavity_faces.shape)
    largest_face_indices = np.where(np.all(largest_face_mask, axis=1))[0]
    largest_cavity_faces = cavity_faces[largest_face_indices]
    
    # Remap face indices
    remapped_faces = np.zeros_like(largest_cavity_faces)
    for i in range(largest_cavity_faces.shape[0]):
        for j in range(3):
            remapped_faces[i, j] = index_map[largest_cavity_faces[i, j]]
    
    # Create and return the largest cavity mesh
    largest_cavity_mesh = o3d.geometry.TriangleMesh()
    largest_cavity_mesh.vertices = o3d.utility.Vector3dVector(vertices[largest_cavity_indices])
    largest_cavity_mesh.triangles = o3d.utility.Vector3iVector(remapped_faces)
    largest_cavity_mesh.compute_vertex_normals()
    
    # Set color for visualization
    cavity_colors = np.ones((len(largest_cavity_indices), 3)) * [0, 1, 0]  # Green
    largest_cavity_mesh.vertex_colors = o3d.utility.Vector3dVector(cavity_colors)
    
    return largest_cavity_mesh, largest_cavity_indices

In [48]:
def extract_cavity_bottom(largest_cavity_mesh, threshold_percentage=0.1):
    # Get vertices and triangles from the mesh
    cavity_vertices = np.asarray(largest_cavity_mesh.vertices)
    cavity_triangles = np.asarray(largest_cavity_mesh.triangles)
    
    # Calculate z-range
    min_z = np.min(cavity_vertices[:, 2])
    max_z = np.max(cavity_vertices[:, 2])
    z_range = max_z - min_z
    
    # Define a threshold for what constitutes the "bottom"
    # Here we're considering the bottom 10% of the cavity's depth
    z_threshold = min_z + z_range * threshold_percentage
    
    # Find vertices that are in the bottom region
    bottom_vertex_mask = cavity_vertices[:, 2] <= z_threshold
    bottom_vertex_indices = np.where(bottom_vertex_mask)[0]
    
    # Find triangles where all three vertices are in the bottom region
    bottom_triangles_mask = np.isin(cavity_triangles.ravel(), bottom_vertex_indices).reshape(cavity_triangles.shape)
    bottom_triangle_indices = np.where(np.all(bottom_triangles_mask, axis=1))[0]
    
    if len(bottom_triangle_indices) == 0:
        print("No triangles found in the bottom region. Try adjusting the threshold.")
        return None
    
    # Create a new mesh for the bottom surface
    bottom_triangles = cavity_triangles[bottom_triangle_indices]
    
    # Get unique vertices used in the bottom triangles
    unique_vertices = np.unique(bottom_triangles.ravel())
    
    # Create index mapping
    index_map = np.zeros(len(cavity_vertices), dtype=int)
    for i, idx in enumerate(unique_vertices):
        index_map[idx] = i
    
    # Remap triangle indices
    remapped_triangles = np.zeros_like(bottom_triangles)
    for i in range(bottom_triangles.shape[0]):
        for j in range(3):
            remapped_triangles[i, j] = index_map[bottom_triangles[i, j]]
    
    # Create bottom surface mesh
    bottom_mesh = o3d.geometry.TriangleMesh()
    bottom_mesh.vertices = o3d.utility.Vector3dVector(cavity_vertices[unique_vertices])
    bottom_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)
    bottom_mesh.compute_vertex_normals()
    
    # Set color for visualization
    bottom_colors = np.ones((len(unique_vertices), 3)) * [0, 0, 1]  # Blue for bottom surface
    bottom_mesh.vertex_colors = o3d.utility.Vector3dVector(bottom_colors)
    
    return bottom_mesh

In [49]:
def calculate_surface_roughness(mesh):
    # area ratio method
    """Calculate various roughness metrics for a mesh surface """
    # 3. Surface area based roughness
    # Calculate the total surface area
    mesh_area = mesh.get_surface_area()
    
    # Calculate the projected area on the XY plane
    # This is an estimation assuming bottom surface is mostly horizontal
    # Create a convex hull in 2D
    points_2d = vertices[:, 0:2]  # Just x,y coordinates
    from scipy.spatial import ConvexHull
    hull = ConvexHull(points_2d)
    projected_area = hull.volume  # In 2D, volume is actually area
    
    print(f"projected_area : {projected_area}")
    print(f"mesh_area : {projected_area}")

    # Surface area ratio (SAR)
    area_ratio = mesh_area / projected_area if projected_area > 0 else float('inf') # Gerçek alanın izdüşüm alanına oranı
    
    # area ratio method
    
    return area_ratio

In [50]:
def visualize_roughness(mesh):
    vertices = np.asarray(mesh.vertices)
    z_values = vertices[:, 2]
    z_mean = np.mean(z_values)
    
    # Calculate deviations from mean
    deviations = np.abs(z_values - z_mean)
    max_deviation = np.max(deviations)
    
    # Create color map based on deviations
    # Blue (0,0,1) for flat areas, red (1,0,0) for rough areas
    colors = np.zeros((len(vertices), 3))
    normalized_deviations = deviations / max_deviation if max_deviation > 0 else np.zeros_like(deviations)
    
    colors[:, 0] = normalized_deviations  # Red channel
    colors[:, 2] = 1.0 - normalized_deviations  # Blue channel
    
    # Apply colors to mesh
    colored_mesh = o3d.geometry.TriangleMesh()
    colored_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    colored_mesh.triangles = mesh.triangles
    colored_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    colored_mesh.compute_vertex_normals()
    
    return colored_mesh

In [52]:

cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
outline_indices = np.where((mean_curvature > 3.0))[0] 
largest_cavity_mesh, largest_cavity_indices = extract_largest_cavity(vertices, faces, cavity_indices)

largest_cavity_vertices= np.asarray(largest_cavity_mesh.vertices)


cavity_bottom = extract_cavity_bottom(largest_cavity_mesh, threshold_percentage=0.4)

# kavite altının z eksenindeki ortalamasını al
bottom_vertices = np.asarray(cavity_bottom.vertices)

bottom_z_values = bottom_vertices[:, 2]
min_z_mean = np.mean(bottom_z_values) 

max_z = np.max(largest_cavity_vertices[:, 2])
cavity_depth = max_z - min_z_mean  # Derinlik (Z eksenindeki fark)

cavity_centroid = np.mean(largest_cavity_vertices, axis=0)


min_z_point = [cavity_centroid[0], cavity_centroid[1], min_z_mean]
max_z_point = [cavity_centroid[0], cavity_centroid[1], max_z]
area_ratio = calculate_surface_roughness(cavity_bottom)

roughness_mesh = visualize_roughness(cavity_bottom) ## visualizing roughness



# **Çizgiyi tanımlama**
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector([min_z_point, max_z_point])
line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0]])  # Mavi çizgi

print(f"kavite derinliği : {cavity_depth}")
print(f"kavite yüzeyinin pürüzlülüğ( area_ratio) : {area_ratio}")
# o3d.visualization.draw_geometries([line_set])




# # Visualize
# # Get unique cavity vertices and their indices
# unique_cavity_indices = np.unique(cavity_indices)
# cavity_vertices = vertices[unique_cavity_indices]
# # Create a mapping from original vertex indices to new vertex indices
# index_map = np.zeros(len(vertices), dtype=int)
# for i, idx in enumerate(unique_cavity_indices):
#     index_map[idx] = i


mesh_o3d = o3d.geometry.TriangleMesh()
mesh_o3d.vertices = o3d.utility.Vector3dVector(vertices)
mesh_o3d.triangles = o3d.utility.Vector3iVector(faces)
mesh_o3d.compute_vertex_normals()

colors = np.ones((vertices.shape[0], 3)) * 0.7  # Light gray for normal surface
colors[outline_indices] = [1, 0, 0]  # Red for cavities
colors[cavity_indices] = [0, 1, 0]  # Red for cavities
mesh_o3d.vertex_colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([mesh_o3d])

# # Create a new mesh for just the cavity portion
# cavity_mesh = o3d.geometry.TriangleMesh()


# # Find faces that have all vertices in cavity_indices
# cavity_face_mask = np.isin(faces.ravel(), unique_cavity_indices).reshape(faces.shape)
# cavity_face_indices = np.where(np.all(cavity_face_mask, axis=1))[0]
# cavity_faces = faces[cavity_face_indices]


projected_area : 109.42617638376166
mesh_area : 109.42617638376166
kavite derinliği : 2.0301277332356844
kavite yüzeyinin pürüzlülüğ( area_ratio) : 0.17064884779484008
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


KeyboardInterrupt: 